### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/home')

In [ ]:
import sys

### DIR and DEPENDENCIES

In [ ]:
DIR = '/content/home/Shareddrives/WATRS_Field_Data/Student_Projects/B_SanLuis'
#DIR = '/content/home/MyDrive/B_SanLuis'
sys.path.append(DIR + '/Scripts')

CropConcern = 'Broccoli2023' #Change between Broccoli and Lettuce
latitude, longitude = 36.4593, -121.3877 #tower location
hemisphere = 'north'

if CropConcern == 'Broccoli2023' :
  in_df = DIR + '/Data/1_Intermediate_Datasets/Broccoli/Broccoli2023FP.csv'
  Json_out_df = DIR + '/Data/1_Intermediate_Datasets/Broccoli/Json/'
  Tiff_out_df = DIR + '/Data/1_Intermediate_Datasets/Broccoli/Tiff/'

if CropConcern == 'Lettuce2023' :
  in_df = DIR + '/Data/1_Intermediate_Datasets/Lettuce/Lettuce2023FP.csv'
  Json_out_df = DIR + '/Data/1_Intermediate_Datasets/Lettuce/Json/'
  Tiff_out_df = DIR + '/Data/1_Intermediate_Datasets/Lettuce/Tiff/'

if CropConcern == 'Lettuce2024' :
  in_df = DIR + '/Data/1_Intermediate_Datasets/Lettuce/Lettuce2024FP.csv'
  Json_out_df = DIR + '/Data/1_Intermediate_Datasets/Lettuce/Json/'
  Tiffout_df = DIR + '/Data/1_Intermediate_Datasets/Lettuce/Tiff/'


In [ ]:
!pip install rasterio
# !pip install memory_profiler

# #%load_ext memory_profiler

In [5]:
import pandas as pd
import geopandas as gpd
import calc_footprint_FFP as ffp
import numpy as np
from shapely.ops import unary_union
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.geometry import shape
from pyproj import Transformer
import json
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
from rasterio.features import shapes
from rasterio import features
from rasterio.plot import show
import matplotlib.pyplot as plt

import gc

###JSon ForLoop

In [6]:
towercoordinates = np.array([[latitude, longitude]]) #create numpy array of latitudes and longitudes
utmzone = int((longitude + 180) / 6) + 1 #calculate UTM zone based on longitude

utm_crs = f"+proj=utm + zone = {utmzone} + {hemisphere} + datum = WGS84 + units = m + no_defs" #create UTM CRS based on longitude and hemisphere
transformer = Transformer.from_crs("epsg:4326", utm_crs)

def convert_to_utm(lat_lon): #creates a function that converts lon/lat to utm
    lat, lon = lat_lon
    easting, northing = transformer.transform(lat, lon)
    return easting, northing #meters from bottom left of utmzone

tower_utm_coordinates = np.apply_along_axis(convert_to_utm, axis = 1, arr = towercoordinates) #calls function and applies to each row in array
easting, northing = tower_utm_coordinates[0]

In [7]:
crop_df = pd.read_csv(in_df) #read in csv (has 1048 rows)
gdf = gpd.GeoDataFrame(columns = ['geometry'], crs = utm_crs) #creating a geodataframe

In [ ]:
import gc

batch_size = 2  # Adjust this number based on your system's capacity

num_rows = len(crop_df)
for batch_start in range(0, num_rows, batch_size):
    batch_end = min(batch_start + batch_size, num_rows)
    batch_df = crop_df.iloc[batch_start:batch_end]

    for index, row in batch_df.iterrows():
        try:
            TestFPmodel = ffp.FFP(
                zm = row['zm'],
                z0 = row['z0'],
                umean = row['u_mean'],
                h = 50,  # Consider reducing this value if possible
                ol = row['L'],
                sigmav = row['sigma_v'],
                ustar = row['u_star'],
                wind_dir = row['wind_dir'],
                rs = [90],
                fig = 0
            )

            # Limit the number of points to reduce memory usage
            xr = np.array(TestFPmodel['xr'][0]) + easting
            yr = np.array(TestFPmodel['yr'][0]) + northing

            max_points = 300  # Adjust as needed
            if len(xr) > max_points:
                indices = np.linspace(0, len(xr)-1, max_points, dtype=int)
                xr = xr[indices]
                yr = yr[indices]

            coordinatepairs = Polygon(zip(xr, yr))

            polygon = gpd.GeoDataFrame(index=[0], crs=utm_crs, geometry=[coordinatepairs])

            Json_output_path = Json_out_df + CropConcern + '_' + str(index) + '.geojson'
            polygon.to_file(Json_output_path, driver='GeoJSON')

            print(CropConcern + f" Polygon {index} saved as {Json_output_path}")

            # Clean up variables to free memory
            del TestFPmodel, xr, yr, coordinatepairs, polygon
            gc.collect()

        except Exception as e:
            print(f"Error in row {index}: {e}")
            continue



Error(0009):
 ustar (friction velocity) must be >=0.1.

Alert(0013):
 Using z0, ignoring umean if passed.
Broccoli2023 Polygon 0 saved as /content/home/Shareddrives/WATRS_Field_Data/Student_Projects/B_SanLuis/Data/1_Intermediate_Datasets/Broccoli/Json/Broccoli2023_0.geojson

Alert(0013):
 Using z0, ignoring umean if passed.
Broccoli2023 Polygon 1 saved as /content/home/Shareddrives/WATRS_Field_Data/Student_Projects/B_SanLuis/Data/1_Intermediate_Datasets/Broccoli/Json/Broccoli2023_1.geojson

Alert(0013):
 Using z0, ignoring umean if passed.
Broccoli2023 Polygon 2 saved as /content/home/Shareddrives/WATRS_Field_Data/Student_Projects/B_SanLuis/Data/1_Intermediate_Datasets/Broccoli/Json/Broccoli2023_2.geojson

Alert(0013):
 Using z0, ignoring umean if passed.
Broccoli2023 Polygon 3 saved as /content/home/Shareddrives/WATRS_Field_Data/Student_Projects/B_SanLuis/Data/1_Intermediate_Datasets/Broccoli/Json/Broccoli2023_3.geojson

Alert(0013):
 Using z0, ignoring umean if passed.
Broccoli2023 

In [ ]:
'''


# for index, row in crop_df.iterrows():  #<-- for entire dataset, crashes: not enough ram
#   try:

for index, (i, row) in enumerate(crop_df.iterrows()): # <-- for only specified amount of rows
#  if index >= 300:
#    break

  try:
    TestFPmodel = ffp.FFP (
      zm = row['zm'],
      z0 = row['z0'],
      umean = row['u_mean'],
      h = 2000,
      ol = row['L'],
      sigmav = row['sigma_v'],
      ustar = row['u_star'],
      wind_dir = row['wind_dir'],
      rs = [90],
      fig = 0 #change to 1 if want to see figure
    )

    xr = np.array(TestFPmodel['xr'][0]) + easting #extracts only the xr and yr and adds the meters of tower location
    yr = np.array(TestFPmodel['yr'][0]) + northing

    coordinatepairs = Polygon(zip(xr, yr)) #zips xr and yr to coordinate pairs together

    polygon = gpd.GeoDataFrame(index = [0], crs = utm_crs, geometry = [coordinatepairs]) #saves the zipped pairs as a geodataframe called polygon
    polygon = polygon.to_crs(utm_crs) # sets the coordinate system

    gdf = pd.concat([gdf, polygon], ignore_index = True) #adds the coordinatepairs to the geodataframe created earlier

    Json_output_path = Json_out_df + CropConcern + '_' + str(index) + '.geojson' #saves to designated folder
    #polygon.to_file(Json_output_path, driver='GeoJSON')

    print(CropConcern + f"Polygon {index} saved as {Json_output_path}")

  except:
    print(f"Error in row {index}") #if their is error, it will skip it and continue
    continue
'''

###FINAL

In [ ]:
# Check if there is SOMETHING in gdf
if not gdf.empty:
    overall_extent = gdf.unary_union.bounds
    minx, miny, maxx, maxy = overall_extent #find furthest extent of polygons

    print(f"Overall extent: minx = {minx}, miny = {miny}, maxx = {maxx}, maxy = {maxy}")
else:
    print("No polygons were processed.")

In [ ]:
# input desired resolution
resolution = 1
width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)

# create array that will hold all individual rasters created in loop
accumulated_raster = np.zeros((height, width), dtype=np.uint8)

transform = from_origin(minx, maxy, resolution, resolution)

In [ ]:

# turn all polygons into rasters and save into int. data + new array
for index, row in gdf.iterrows():
    try:
        polygon = row['geometry']
        geom = [polygon]

        rasterized = features.rasterize(
            geom,
            out_shape=(height, width),
            fill=0,
            transform=transform
        )

        accumulated_raster += rasterized

        # print(f"Rasterized data for polygon {index}:")
        # print(f"Shape: {rasterized.shape}")
        # print(f"Unique values: {np.unique(rasterized)}")
        # print(f"Non-zero values count: {np.count_nonzero(rasterized)}")

        # saves individual rasters in int data folder as tif (saves as black screen currently)
        Tiff_output_path = f"{Tiff_out_df}{CropConcern}_{index}.tif"
        with rasterio.open(
            Tiff_output_path,
            'w',
            driver = 'GTiff',
            height = height,
            width = width,
            count = 1,
            dtype = rasterized.dtype,
            crs = utm_crs,
            transform = transform
        ) as dst:
            dst.write(rasterized, 1)
            # print(f"{CropConcern} Polygon {index} saved as {Tiff_output_path}")

    except Exception as e:
        print(f"Error in row {index}: {e}")
        continue


In [ ]:
# Plot rasters with colormap
fig, ax = plt.subplots()
cax = ax.imshow(accumulated_raster, cmap = 'hot', extent = (minx, maxx, miny, maxy)) #cant decide what colormap to use
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.colorbar(cax, ax = ax, label = 'Overlap Count')
plt.title('Accumulated Raster with Overlap')
plt.show()


In [ ]:
accumulated_raster

###Rid of zeroes

In [ ]:
non_zero_pixels = np.where(accumulated_raster > 0) # identifies all of the pixels that have a value greater than 1
non_zero_values = accumulated_raster[non_zero_pixels] # actually extracts these pixels from accumulated raster

xs, ys = rasterio.transform.xy(transform, non_zero_pixels[0], non_zero_pixels[1]) # pairs the pixels with their corresponding coordinates

geometries = [Point(x, y) for x, y in zip(xs, ys)] # creates a point for each coordinate pair that the pixel will represent

accumulated_raster_non_zero = gpd.GeoDataFrame({ #creates new geodataframe with the geometry (location and size) and the # of overlaps it has
    'geometry': geometries,
    'overlap_count': non_zero_values
}, crs = utm_crs)

In [ ]:
non_zero_pixels # check

In [ ]:
accumulated_raster_non_zero # check

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
accumulated_raster_non_zero.plot(
    ax = ax,
    column = 'overlap_count',
    cmap = 'hot',
    legend = True,
    legend_kwds = {'label': "Overlap Count"}
)

ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.title('Non-Zero Overlap Pixels')
plt.show()

###Percentile

In [ ]:
np.percentile(accumulated_raster, [50,100]) # check percentile that includes 0's

In [ ]:
np.percentile(accumulated_raster_non_zero['overlap_count'], [40,100]) # check percentile that excludes 0's

In [ ]:
percentile_pixels = np.where(accumulated_raster > 7) # identifies all of the pixels that have a value greater than 1
percentile_values = accumulated_raster[percentile_pixels] # actually extracts these pixels from accumulated raster

xs, ys = rasterio.transform.xy(transform, percentile_pixels[0], percentile_pixels[1]) # pairs the pixels with their corresponding coordinates

percentile_geometries = [Point(x, y) for x, y in zip(xs, ys)] # creates a point for each coordinate pair that the pixel will represent

accumulated_raster_percentile = gpd.GeoDataFrame({ #creates new geodataframe with the geometry (location and size) and the # of overlaps it has
    'geometry': percentile_geometries,
    'overlap_count': percentile_values
}, crs=utm_crs)

In [ ]:
percentile_pixels # check

In [ ]:
accumulated_raster_percentile # check

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
accumulated_raster_percentile.plot(
    ax = ax,
    column = 'overlap_count',
    cmap = 'hot',
    legend = True,
    legend_kwds = {'label': "Overlap Count"}
)

ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.title('Non-Zero Overlap Pixels')
plt.show()

###Raster to Vector


In [ ]:
accumulated_raster_percentile

In [ ]:
def raster_to_polygon(raster, transform):
  mask = raster > 7 # this also does the masking based on # of overlap so I can get rid of code above
  shapes_gen = shapes(raster, mask = mask, transform = transform) # generates the shapes from the raster data
  raster_to_polygons = [shape(s[0]) for s in shapes_gen] # turns the shapes into a polygon
  return raster_to_polygons

In [ ]:
raster_to_polygons = raster_to_polygon(accumulated_raster, transform) # turns rasters to polygons... surprise

combined_vector_polygons = unary_union(raster_to_polygons) # merges multiple things into one

vector_gdf = gpd.GeoDataFrame(geometry = [combined_vector_polygons], crs = utm_crs) # turns the merged polygons into a gdf

In [ ]:
vector_gdf # check

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
vector_gdf.plot(ax = ax, edgecolor = 'black', facecolor = 'none')
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.title('Vector Polygon')